Exploring the how reagent contribute to gene counts. 

In [1]:
# %load ../start.py
# Load useful extensions
import os
import sys

# Activate the autoreload extension for easy reloading of external packages
%reload_ext autoreload
%autoreload 1

# Set up cashdir
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Trun on the water mark
%reload_ext watermark
%watermark -u -d -v -g

# Load ipycache extension
%reload_ext ipycache
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Add project library to path
sys.path.insert(0, '../../lcdb-wf/lib')
sys.path.insert(0, '../../lib/python')

# Set up references
import yaml
with open('../../config/config.yml') as fh:
    config = yaml.load(fh)

assembly = config['assembly']
tag = config['aligner']['tag']
REF = os.path.join(os.environ['REFERENCES_DIR'], assembly, tag)


last updated: 2017-09-11 

CPython 3.5.2
IPython 6.1.0
Git hash: 30323ff5d7b851313f09f17f572ce19c6c6afe6c


In [2]:
gene = 'Adf1'

In [200]:
# imports
import collections
import pandas as pd
import pybedtools
import HTSeq

In [22]:
# import sample table
sampletable = pd.read_table('../../config/sampletable.tsv')
mask = sampletable.target_symbol == gene
srrs = sampletable[mask].samplename.unique().tolist()
drsc = sampletable[mask].drsc.unique().tolist()
fbgn = sampletable[mask].target_FBgn.unique().tolist()

sampletable[mask]

,samplename,SRX,BioSample,GEO,drsc,target_FBgn,target_symbol,drsc_rep,rep,plate_id,well_id,plate_row,plate_column
948,SRR3486828,SRX1748824,SAMN04959159,GSM2145091,DRSC30750,FBgn0284249,Adf1,1,2,10,H4,H,4
949,SRR3486733,SRX1748729,SAMN04959113,GSM2144996,DRSC30750,FBgn0284249,Adf1,1,1,10,H4,H,4


In [346]:
# import drsc coords and make drsc_bed
bed = pd.read_table('../../output/drsc_coordinates.bed', header=None)
bed.columns = 'chrom', 'start', 'end', 'rname', 'score', 'strand'
bed.chrom = 'chr' + bed.chrom
mask = bed.rname.isin(drsc)

drsc_bed = pybedtools.BedTool(bed[mask].values.tolist())

In [575]:
# Import GTF and make gene_bed
def featuretype_filter(feature, featuretype, fbgns):
    if (feature[2] == featuretype) & (feature.attrs['gene_id'] in fbgns):
        return True
    return False

assembly = config['assembly']
tag = config['gtf']['tag']
gtfName = os.path.join(
    os.environ['REFERENCES_DIR'], assembly, tag, 'gtf', '{assembly}_{tag}.gtf'.format(
        assembly=assembly, tag=tag
    )
)

with open(gtfName) as fh:
    gtf_full = pybedtools.BedTool(fh.read().strip(), from_string=True)
gene_bed = pybedtools.BedTool(gtf_full.filter(featuretype_filter, 'exon', fbgn).saveas().fn)

In [398]:
# Subtract DRSC from Gene model
gene_sub = gene_bed.subtract(drsc_bed)

In [474]:
# Build gene interval
gtf_fh = HTSeq.GFF_Reader(gene_bed.fn)
gene_interval = HTSeq.GenomicArrayOfSets("auto", stranded=True)
for feature in gtf_fh:
    if (feature.type == 'exon') & (feature.attr['gene_id'] in fbgn):
        gene_interval[feature.iv] += feature.attr['gene_id']

In [475]:
# Build gene drsc subtracted interval
gtf_fh = HTSeq.GFF_Reader(gene_sub.fn)
gene_sub_interval = HTSeq.GenomicArrayOfSets("auto", stranded=True)
for feature in gtf_fh:
    if (feature.type == 'exon') & (feature.attr['gene_id'] in fbgn):
        gene_sub_interval[feature.iv] += feature.attr['gene_id']

In [476]:
# Build drsc interval
drsc_interval = HTSeq.GenomicArrayOfSets("auto", stranded=True)
for feature in drsc_bed:
    interval = HTSeq.GenomicInterval(
        chrom=feature.chrom,
        start=feature.start,
        end=feature.end,
        strand=feature.strand
    )
    drsc_interval[interval] += feature.name

In [425]:
# Get BAMs file names
bams = []
for srr in srrs:
    bams.append(os.path.join(
        '../../rnaseq-wf/data/rnaseq_samples/{srr}/{srr}.cutadapt.bam'.format(srr=srr)
    ))

bams

['../../rnaseq-wf/data/rnaseq_samples/SRR3486828/SRR3486828.cutadapt.bam',
 '../../rnaseq-wf/data/rnaseq_samples/SRR3486733/SRR3486733.cutadapt.bam']

In [49]:
def invert_strand(iv):
    iv2 = iv.copy()
    if iv2.strand == "+":
        iv2.strand = "-"
    elif iv2.strand == "-":
        iv2.strand = "+"
    else:
        raise ValueError("Illegal strand")
    return iv2

In [477]:
counter = {
    'gene': 0,
    'sub': 0,
    'drsc': 0
}

almnt_file = HTSeq.BAM_Reader(bams[0])
for almnt in almnt_file:
    if not almnt.aligned:
        continue
        
    # skip reads that are on different chromosomes
    if almnt.iv.chrom not in list(gene_interval.chrom_vectors.keys()):
        continue
        
    # Count on full gene
    gene_ids = set()
    for iv, val in gene_interval[invert_strand(almnt.iv)].steps():
        gene_ids |= val
        
    if len(gene_ids) == 1:
        gene_id = list(gene_ids)[0]
        counter['gene'] += 1
        
    # Count on drsc subtracted gene
    gene_ids = set()
    for iv, val in gene_sub_interval[invert_strand(almnt.iv)].steps():
        gene_ids |= val
        
    if len(gene_ids) == 1:
        gene_id = list(gene_ids)[0]
        counter['sub'] += 1
        
    # Count on drsc 
    gene_ids = set()
    for iv, val in drsc_interval[invert_strand(almnt.iv)].steps():
        gene_ids |= val
        
    if len(gene_ids) == 1:
        gene_id = list(gene_ids)[0]
        counter['drsc'] += 1

In [479]:
gene_counts

Counter({'FBgn0284249': 460})

In [480]:
gene_sub_counts

Counter({'FBgn0284249': 296})

In [478]:
drsc_counts

Counter({'DRSC30750': 178})

In [500]:
counter = {'gene': 460, 'sub': 296, 'drsc': 178}

dict_values([460])

In [539]:
df = pd.DataFrame([counter], index=['SRR'])

In [533]:
def get_bed_len(bed):
    """Calcuates the total length of a gene.

    Returns
    -------
    int:
        Length of a gene.
    """
    length = 0
    merged = bed.sort().merge()
    for x in merged:
        length += x.length
    return length

In [534]:
get_bed_len(gene_bed)

1800

In [536]:
get_bed_len(gene_sub)

1572

In [537]:
get_bed_len(drsc_bed)

228

In [545]:
df.index.name='srr'
print(df.reset_index().to_string(index=False))

srr  drsc  gene  sub
SRR   178   460  296


In [546]:
print(gene_bed)

chr2R	FlyBase	exon	6662265	6662436	7	+	.	gene_symbol "Adf1";transcript_id "FBtr0086111";gene_id "FBgn0284249";transcript_symbol "Adf1-RA";
chr2R	FlyBase	exon	6662569	6662757	7	+	.	gene_symbol "Adf1";transcript_id "FBtr0086111";gene_id "FBgn0284249";transcript_symbol "Adf1-RA";
chr2R	FlyBase	exon	6666720	6667013	7	+	.	gene_symbol "Adf1";transcript_id "FBtr0086111";gene_id "FBgn0284249";transcript_symbol "Adf1-RA";
chr2R	FlyBase	exon	6667073	6668085	7	+	.	gene_symbol "Adf1";transcript_id "FBtr0086111";gene_id "FBgn0284249";transcript_symbol "Adf1-RA";
chr2R	FlyBase	exon	6662270	6662757	15	+	.	gene_symbol "Adf1";transcript_id "FBtr0086112";gene_id "FBgn0284249";transcript_symbol "Adf1-RB";
chr2R	FlyBase	exon	6666720	6667013	15	+	.	gene_symbol "Adf1";transcript_id "FBtr0086112";gene_id "FBgn0284249";transcript_symbol "Adf1-RB";
chr2R	FlyBase	exon	6667073	6668085	15	+	.	gene_symbol "Adf1";transcript_id "FBtr0086112";gene_id "FBgn0284249";transcript_symbol "Adf1-RB";
chr2R	FlyBase	exon	66622